# generating logs from the particle model
Tim Tyree<br>
7.1.2021

In [ ]:
from lib.my_initialization import *
from lib.utils.pbc import *
import shutil
from numpy.random import MT19937
from numpy.random import RandomState, SeedSequence

from lib.routines.gener_log import *

%load_ext autoreload
%autoreload 2
get_routine_gener_logs

In [2]:
#DONE: dev functional routine that takes an integer seed as input and outputs a directory of the resulting _log.csv file
#DONE: put ^that in a .py file and load it
#DONE: make a list of enough integers
# seed_values=np.arange(100)+1
#DONE: daskbag over ^that


In [ ]:
# #performace test
# results_folder=f"{nb_dir}/data/local_results"
# routine_gener_logs=get_routine_gener_logs(results_folder,tmax=.1)
# routine_gener_logs(seed=1)
# 16*100/60 #=~27min per trial
Dt=1e-5
dt=1e-5
iter_per_movestep = round(Dt / dt)
iter_per_movestep

In [22]:
#on Mac...
# 16*100/60 #=~27min per trial
#8 cores... 12 hours...
# 12*60*8/27 #213 trials
num_trials=300
seed_values=np.arange(num_trials)+1
tmax=100
varkappa=5
D=0.7
kappa=1500
r=0.1
save_every=25
use_neighbors=1
N=2#16
set_pair_ic=True

In [28]:
D_values=np.arange(0,2.1,.1,)


In [29]:
# #TODO: dev 
# # L=1 set_pair_ic=True
# x_values=np.array([0,0.5])
# y_values=np.array([0,0.5])
# np.sqrt(sum((x_values-y_values)**2))#They're touching

In [37]:
D_values,npartitions

(array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2,
        1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. ]),
 12)

In [38]:
DT=save_every*Dt #seconds between two frames
print(f"the resulting logs will have a time resolution of {DT*10**3} 'ms' between two frames")

the resulting logs will have a time resolution of 0.25 'ms' between two frames


In [39]:
results_folder=f"{nb_dir}/data/local_results"
printing=True
for D in D_values:
    kwargs=dict(tmax=tmax,varkappa=varkappa,D=D,kappa=kappa,use_neighbors=use_neighbors,r=r,N=N,set_pair_ic=set_pair_ic)
    routine_gener_logs=get_routine_gener_logs(results_folder,**kwargs)
    routine=routine_gener_logs
    npartitions=os.cpu_count()
    #for each trial set...
    bag = db.from_sequence(seed_values, npartitions=npartitions).map(routine)
    if printing:
        print(f"computing xy trajectories for the particle model with D={D:.1f} cm^2/s...\nconsidering in {len(list(seed_values))} trial sets.\noutputing to .csv in results_folder={results_folder}...")
    start = time.time()
    retval_lst = list(bag)
    if printing:
        print(f"the run time for computing xy trajectories was {(time.time()-start)/60:.2f} minutes.")
        

computing xy trajectories for the particle model with D=0.0 cm^2/s...
considering in 300 trial sets.
outputing to .csv in results_folder=/home/timothytyree/Documents/GitHub/bgmc/python/data/local_results...


Process SpawnProcess-46:
Traceback (most recent call last):
  File "/home/timothytyree/anaconda3/envs/pyenv_ub/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/timothytyree/anaconda3/envs/pyenv_ub/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/timothytyree/anaconda3/envs/pyenv_ub/lib/python3.7/concurrent/futures/process.py", line 233, in _process_worker
    call_item = call_queue.get(block=True)
  File "/home/timothytyree/anaconda3/envs/pyenv_ub/lib/python3.7/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/home/timothytyree/anaconda3/envs/pyenv_ub/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
Process SpawnProcess-44:
Traceback (most recent call last):
  File "/home/timothytyree/anaconda3/envs/pyenv_ub/lib/python3.7/multiprocessing/process.py", line 297, in _bootstra

KeyboardInterrupt: 

In [33]:
# results_folder=f"{nb_dir}/data/local_results"
# kwargs=dict(tmax=tmax,varkappa=varkappa,D=D,kappa=kappa,use_neighbors=use_neighbors,r=r,N=N,set_pair_ic=set_pair_ic)
# routine_gener_logs=get_routine_gener_logs(results_folder,**kwargs)
# routine=routine_gener_logs
# npartitions=os.cpu_count()
# #for each trial set...
# bag = db.from_sequence(seed_values, npartitions=npartitions).map(routine)
# print(f"computing xy trajectories for the particle model...\nconsidering in {len(list(seed_values))} trial sets.\noutputing to .csv in results_folder={results_folder}...")
# start = time.time()
# retval_lst = list(bag)
# print(f"the run time for computing xy trajectories was {(time.time()-start)/60:.2f} minutes.")

In [ ]:
300/60

On my Mac, 200 trials took ~3 hours at a temporal resolution of DT=0.25 ms between two frames (using all 8 "cores" shared with 8 other python processes...).

On my UB, 200 trials took ~2, 1.33 hours at a temporal resolution of DT=0.25 ms between two frames (using all 12 cores)

On my UB, 50 ~2, 1.33 hours at a temporal resolution of DT=0.25 ms between two frames (using all 12 cores)

In [ ]:

#scratchwork
#define model parameters baked into simulation
# N=16
# L=10 #cm
# r=0.1 #cm
# D=2 #cm^2/s
# kappa=1500 #Hz
# # Dt=1e-5;dt=1e-5;tmax=10 #s #UNCOMMENT_HERE
# Dt=1e-5;dt=1e-5;tmax=.1 #s #COMMENT_HERE
# force_code=2
# varkappa=5
# save_every=25#frames
# dist_cutoff=1e-2#cm
# x0=5
# no_repulsion=1
# no_attraction=0

#for forces
# # message='without forces'
# # force_code=0
# # varkappa=0
# message='with anti-spring forces'
# force_code=1
# varkappa=-5
# message='with inverse powerlaw forces'
# force_code=2
# varkappa=5

__TODO__: with tmax=3 and kappa=0 and varkappa=0, generate a folder of particle locations... Could this be an artifact of particles switching places?  This could be in how I'm doing the tracking... 
__TODO__: See if the trackpy particle tracking is to blame by explicitely tracking particles in the routine

```printing summary stats on particle lifetimes for one input folder in /Users/timothytyree/Documents/GitHub/bgmc/python/data/local_results/euic_False_fc_2_r_0.1_D_2_L_10_kappa_1500_varkappa_0:
count       16.000000
mean      2868.140625
std       3345.547204
min          1.250000
25%        444.187500
50%       1740.750000
75%       3737.437500
max      10000.000000
Name: min, dtype: float64
particle
0       57.75
1    10000.00
2     2325.25
3     3059.00
4     9999.75
5        1.25
6     3059.00
7     5772.75
8      573.00
9     5772.75
Name: min, dtype: float64```

# TODO: grid search over D from 0.1 to 2 in steps on 0.14
(GPU accelerate in cupy)
TODO: rewrite it from scratch, developing in vectorized pairs of particles
1. start 2 particles 1/np.sqrt(2) away from one another


__Schema__
1. t,x1,x2,y1,y2,pair_num

In [40]:
1/np.sqrt(2)

0.7071067811865475